In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Dependencies
import torch as th
import torch.nn.functional as F
from torch import nn

use_cuda = False
th.manual_seed(1)
device = th.device("cuda" if use_cuda else "cpu")

import syft as sy
from syft import workers

hook = sy.TorchHook(th)  # hook torch as always :)

In [3]:
sy.__path__

['/home/billy/billy_ws/fl_025/PySyft/syft']

In [4]:
sy.local_worker

<VirtualWorker id:me #objects:0>

In [5]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
@sy.func2plan(args_shape=[(-1, 1), (-1, 1)])
def loss_fn(target, pred):
    return ((target.view(pred.shape).float() - pred.float()) ** 2).mean()

In [8]:
net = Net()
net.build(th.tensor([1., 2.]))

PlaceHolder[Id:27591511241]>tensor([-0.0494], grad_fn=<AddBackward0>)

In [9]:
loss_fn.is_built

True

In [10]:
optimizer = "SGD"
batch_size = 4
optimizer_args = {"lr" : 0.1, "weight_decay" : 0.01}
epochs = 1
max_nr_batches = -1  # not used in this example
shuffle = True

In [11]:
model_config = sy.ModelConfig(model=net,
                              loss_fn=loss_fn,
                              optimizer=optimizer,
                              batch_size=batch_size,
                              optimizer_args=optimizer_args,
                              epochs=epochs,
                              shuffle=shuffle)

## Connect to remote worker

In [12]:
from syft.workers.node_client import NodeClient

In [13]:
alice = NodeClient(hook, "ws://localhost:6666" , id="alice")

In [14]:
ptr_model = sy.local_worker.send(net, alice)

In [19]:
ptr_model = net.send(alice)

In [20]:
ptr_model

Net(
  (fc1): Linear(in_features=2, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=1, bias=True)
)

In [16]:
model_config.send(alice)

TypeError: can not serialize 'ModelConfig' object